In [1]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D

# Define the paths to the clear and foggy image folders
clear_folder = "C:/Users/HP/Pictures/clearFoggy/Clear/"
foggy_folder = "C:/Users/HP/Pictures/clearFoggy/Foggy/"

# Set the desired image size
image_height = 256
image_width = 256
num_channels = 3

# Load the clear and foggy images
clear_images = []
foggy_images = []

for filename in os.listdir(foggy_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        foggy_img = cv2.imread(os.path.join(foggy_folder, filename))
        clear_filename = os.path.splitext(filename)[0] + ".jpg"  # Assuming clear images have .jpg extension
        clear_img = cv2.imread(os.path.join(clear_folder, clear_filename))
        foggy_img = cv2.resize(foggy_img, (image_width, image_height))
        clear_img = cv2.resize(clear_img, (image_width, image_height))
        foggy_images.append(foggy_img)
        clear_images.append(clear_img)

# Convert the lists of images to numpy arrays
clear_images = np.array(clear_images)
foggy_images = np.array(foggy_images)

# Normalize pixel values to range between 0 and 1
clear_images = clear_images.astype('float32') / 255.0
foggy_images = foggy_images.astype('float32') / 255.0

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(1024, (3, 3), activation='relu', padding='same', input_shape=(image_height, image_width, num_channels)))
model.add(Conv2D(1024, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(num_channels, (3, 3), activation='relu', padding='same'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(foggy_images, clear_images, epochs=10, batch_size=5)

# Save the trained model
model.save("foggy_to_clear_model_high_filter.h5")


Epoch 1/10


In [ ]:
import cv2
import numpy as np
from keras.models import load_model

# Set the desired image size
image_height = 256
image_width = 256
num_channels = 3

# Load the trained model
model = load_model("foggy_to_clear_model.h5")

# Load a foggy image for testing
foggy_image_path = "C:/Users/HP/Documents/Semester (summer)/Aim Lab Internship/Fog/foggy-001.jpg"
foggy_image = cv2.imread(foggy_image_path)

# Resize the foggy image to the desired size
foggy_image = cv2.resize(foggy_image, (image_width, image_height))

# Preprocess the foggy image
foggy_image = foggy_image.astype('float32') / 255.0
foggy_image = np.expand_dims(foggy_image, axis=0)

# Use the model to predict the clear image
clear_image = model.predict(foggy_image)

# Rescale the pixel values to the original range
clear_image = clear_image * 255.0
clear_image = clear_image.astype('uint8')

# Reshape the clear image
clear_image = np.squeeze(clear_image, axis=0)

# Display the clear image
cv2.imshow("Clear Image", clear_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

